In [ ]:
# Import Libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load MovieLens Dataset (adjust path or download dataset)
# Dataset link: https://grouplens.org/datasets/movielens/
movies = pd.read_csv("C:\\Users\\sarow\\Downloads\\ml-32m\\movies.csv")  # Contains movieId, title, genres
ratings = pd.read_csv("C:\\Users\\sarow\\Downloads\\ml-32m\\ratings.csv")  # Contains userId, movieId, rating

# Content-Based Filtering Implementation
print("Step 1: Content-Based Filtering")

# Combine title and genres for TF-IDF feature extraction
movies['content'] = movies['title'] + " " + movies['genres']
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['content'])

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Recommendation function
def recommend_content(movie_title, cosine_sim=cosine_sim):
    # Get index of the movie
    idx = movies[movies['title'] == movie_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in sim_scores[1:6]]
    return movies['title'].iloc[top_indices]

# Example recommendation
print("Recommendations based on Content-Based Filtering:")
print(recommend_content("Toy Story (1995)"))

# Collaborative Filtering Implementation
print("\nStep 2: Collaborative Filtering")

# Prepare dataset for Surprise library
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.25)

# Train SVD model
model = SVD()
model.fit(trainset)

# Evaluate model
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)
print(f"RMSE of Collaborative Filtering: {rmse}")

# Collaborative Filtering Recommendation Function
def recommend_collaborative(user_id, model=model, n_recommendations=5):
    # Get all unique movieIds
    all_movie_ids = ratings['movieId'].unique()
    rated_movie_ids = ratings[ratings['userId'] == user_id]['movieId'].unique()
    unrated_movie_ids = [m for m in all_movie_ids if m not in rated_movie_ids]
    
    # Predict ratings for unrated movies
    predictions = [(movie_id, model.predict(user_id, movie_id).est) for movie_id in unrated_movie_ids]
    predictions = sorted(predictions, key=lambda x: x[1], reverse=True)
    top_movie_ids = [p[0] for p in predictions[:n_recommendations]]
    
    # Return movie titles
    return movies[movies['movieId'].isin(top_movie_ids)]['title'].values

# Example recommendation for user with ID 1
print("\nRecommendations based on Collaborative Filtering:")
print(recommend_collaborative(user_id=1))

